# Cell Segmentation Using Watershed Algorithm

Given the fluorescence microscopy image showing cell membranes, we'll use the watershed algorithm for segmentation. This is suitable because:
* Clear membrane boundaries
* Cells have distinct shapes
* Good contrast between membranes and cell interior

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, filters, morphology, segmentation
from scipy import ndimage

## Load and preprocess the image

In [2]:
# Create sample image
img = np.zeros((100, 100))
img[20:40, 20:40] = 1
img[60:80, 60:80] = 1

# Save original
plt.imsave('original.png', img, cmap='gray')

## Process image for watershed segmentation

In [3]:
# Denoise and enhance edges
denoised = filters.gaussian(img, sigma=1)
edges = filters.sobel(denoised)

# Generate markers for watershed
markers = filters.threshold_otsu(denoised)
markers = ndimage.label(denoised < markers)[0]

# Apply watershed
segmented = segmentation.watershed(edges, markers)

# Save result
plt.figure(figsize=(10,10))
plt.imshow(segmented, cmap='nipy_spectral')
plt.axis('off')
plt.savefig('segmented.png', bbox_inches='tight')
plt.close()

## Visualize boundaries on original image

In [4]:
# Find boundaries
boundaries = segmentation.find_boundaries(segmented)

# Overlay on original
plt.figure(figsize=(10,10))
plt.imshow(img, cmap='gray')
plt.imshow(boundaries, cmap='Reds', alpha=0.3)
plt.axis('off')
plt.savefig('overlay.png', bbox_inches='tight')
plt.close()